# Model Training Code Using Exsisting models

In [16]:
#Importing Necessary Libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import os

In [17]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
#INIT_LR = 1e-4 #default value of adam optimizer
#EPOCHS = 20
#BS = 32

In [18]:
print("[INFO] loading images...")

# To Extract the path of the dataset used for training 
imagePaths = list(paths.list_images("dataset"))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    
    # load the input image with size (299,299) coz, inception_v3 model was build with this input size and as we are
    # doing fine tuning, so we need to give the same size and preprocess it
    image = load_img(imagePath, target_size=(229, 229))
    
    # To convert image into array
    image = img_to_array(image) 
    image = preprocess_input(image)
    
    # appending the data and labels with all files
    data.append(image)
    labels.append(label)

[INFO] loading images...


In [19]:
# Converting the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [20]:
# Perform One-Hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Splitting the data into Trainng & Testing for checking the accuracy with 20% of data for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

In [21]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [22]:
# load the inception_v3 network, ensuring the head FC layer sets are left off
baseModel = inception_v3.InceptionV3(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(229, 229, 3)))

In [23]:
# construct the head of the model that will be placed on top of the the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [24]:
# Actual model which we will train
model = Model(inputs=baseModel.input, outputs=headModel)

# Freezing the layers of baseModel so that it won't be trained again
for layer in baseModel.layers:
    layer.trainable = False

In [25]:
# Compiling our Model using Adam optimiser
print("[INFO] compiling model...")
opt = Adam(lr=1e-4, decay=1e-4 / 10)  #1e-4 is default learning rate for Adam Optimiser
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

[INFO] compiling model...


### Training Our Model
For better accuaracy You have to increase he no. of epochs from 2 to 10. After epoch I got an accuracy of 91% around, You can do this but it will take little time deppending upon the system configuration you have.

In [26]:
# train the head of the network
print("[INFO] training model...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=32),
    steps_per_epoch=len(trainX) // 32,
    validation_data=(testX, testY),
    validation_steps=len(testX) // 32,
    epochs=2)

[INFO] training model...
Train for 34 steps, validate on 276 samples
Epoch 1/2
34/34 [==============================] - 136s 4s/step - loss: 0.6644 - accuracy: 0.6096 - val_loss: 0.2786 - val_accuracy: 0.9531
Epoch 2/2
34/34 [==============================] - 124s 4s/step - loss: 0.4773 - accuracy: 0.7828 - val_loss: 0.1531 - val_accuracy: 0.9727


In [27]:
# Making Predictions on Testing Dataset
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=32)
# for each image in the testing set we need to find the index of the label with corresponding largest 
# predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

   with_mask       0.97      0.98      0.97       138
without_mask       0.98      0.97      0.97       138

    accuracy                           0.97       276
   macro avg       0.97      0.97      0.97       276
weighted avg       0.97      0.97      0.97       276



In [28]:
# Saving our created model so that we can use it for predictions
print("[INFO] saving mask detector model...")
model.save("MaskDetector.h5")

[INFO] saving mask detector model...
